In [1]:
import numpy as np
import math
import pandas as pd
import time

In [2]:
## old implementation of the class. This is slow, do not use!
class FraudAnalyzer:
    """
    Use an externally supplied function to determine the probability that an election result
    is fraudulent.
    """
    def __init__(self, simulation_df, func, patterns=("00", "25"), digits=5):
        """
        Initialize the fraud election detector.
        :param simulation_df: pandas DataFrame. Contains the columns with the votes of each candidate
        :param func: function. Used to generate the closure clos.
                     clos determines if a result is "nice" (i.e. seems fraudulent) based on matching to
                     specified patterns
        :param patterns: tuple of string. Patterns which, combined with digits, determine if a result is "nice"
        :param digits: int. Number of digits by which the decimal point is offset to determine pattern matching
        :return: Nothing
        """
        self.df = simulation_df
        self.clos = func(patterns, digits)

    def find_nice(self):
        """
        Fill in the "is_nice" column of the df attribute
        :return: Nothing
        """
        self.df["is_nice"] = self.df.apply(self.clos, axis=1)

    def fraction_nice(self):
        """
        Determine the fraction of "nice" results in the simulation DataFrame
        :return: float. The fraction of "nice results"
        """
        is_nice = self.df.apply(self.clos, axis=1)
        return float(sum(is_nice))/self.df.shape[0]


    
def generate_is_nice(patterns=("00", "25"), digits=5):
    """
    Generate a closure "is_nice(row)" which takes in a row of numbers and determines
    if the fractional sizes of all numbers share a common simple pattern
    :param patterns: tuple of string. The patterns which determine if a fraction is "nice"
    :param digits: int. Number of digits by which to offset aa fraction.
    :return: closure. The is_nice(row) function returns 1 if the numbers in row give "nice"
            fractions, 0 otherwise.
    """
    def is_nice(row):
        """
        Determine if the
        :param row:
        :return:
        """
        normed_row = [(float(el)/sum(row[:-1])) for el in row[:-1]]
        itis = False
        for pattern in patterns:
            i2 = True
            for num in normed_row:
                i2 = (has_form(num, pattern=pattern, digits=digits) and i2)
            itis = itis or i2
        return float(itis)
    return is_nice


def has_form(num, pattern, digits=5):
    return str(num * 10**digits)[3:5] == pattern

In [21]:
num = 0.39800
print ((num * 10**5) % 100) == 0.0
num = 0.28500
print ((num * 10**5) % 100) == 0.0
ls = [160823, 115162, 82028, 46065]
#ls = [0.39800, 0.28500, 0.20300, 0.11400]
ls = [float(el)/sum(ls) for el in ls]
print [((num * 10**5) % 100) == 0.0 for num in ls]
print [abs((num * 10**5) % 100) <= 0.1 for num in ls]
print [(int(num * 10**5) % 100) == 0 for num in ls]
print [abs(1000*num-int(1000*num)) for num in ls]

arr = np.array([np.float128(num) for num in ls])
print [abs((num * 10**5) % 100) <= 0.1 for num in arr]

True
False
[False, False, False, False]
[False, False, True, True]
[False, False, True, True]
[0.9998911101321823, 0.9994308029637864, 0.00041081177397472857, 0.0002672751300423215]
[False, False, True, True]


In [18]:
s = pd.Series(np.arange(1000,1000000)/np.float64(1000000))
#s = pd.Series(np.array([0.398000, 0.285000, 0.20300, 0.114000]))
t0 = time.time()
s = pd.Series(s.values.astype(np.str))
s_nice = s.apply(lambda e: float(is_nice(e)))
t = time.time()
print t-t0
print s.head()

1.59714889526
0       0.001
1    0.001001
2    0.001002
3    0.001003
4    0.001004
dtype: object


In [17]:
def is_nice(x):
    sub = x.split(".")[1][3:5]
    return (sub == "") or (sub == "00") or (sub == "25")

st = "0.39800"
is_nice(st)

True

In [12]:
print "%5.4f" % s_nice.mean()

0.0000


In [16]:
s_nice.head()

0    1
1    1
2    1
3    1
dtype: float64